In [1]:
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [2]:
print(tf.__version__)

2.4.1


In [3]:
url = "http://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz"

tf.keras.utils.get_file("stack_overflow_16k.tar.gz", url,
                        untar=True, cache_dir='.',
                        cache_subdir='')

6053888/6053168 [==============================] - 0s 0us/step


'./stack_overflow_16k.tar.gz'

In [4]:
batch_size = 32

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'train', batch_size=batch_size, validation_split=0.2, subset='training', seed=42)

raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'train', batch_size=batch_size, validation_split=0.2, subset='validation', seed=42)

raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'test', batch_size=batch_size)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.
Found 8000 files belonging to 4 classes.
Using 1600 files for validation.
Found 8000 files belonging to 4 classes.


In [5]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(5):
    print(text_batch.numpy()[i])
    print(label_batch.numpy()[i])
    print()

b'"my tester is going to the wrong constructor i am new to programming so if i ask a question that can be easily fixed, please forgive me. my program has a tester class with a main. when i send that to my regularpolygon class, it sends it to the wrong constructor. i have two constructors. 1 without perameters..public regularpolygon().    {.       mynumsides = 5;.       mysidelength = 30;.    }//end default constructor...and my second, with perameters. ..public regularpolygon(int numsides, double sidelength).    {.        mynumsides = numsides;.        mysidelength = sidelength;.    }// end constructor...in my tester class i have these two lines:..regularpolygon shape = new regularpolygon(numsides, sidelength);.        shape.menu();...numsides and sidelength were declared and initialized earlier in the testing class...so what i want to happen, is the tester class sends numsides and sidelength to the second constructor and use it in that class. but it only uses the default constructor, w

In [6]:
max_features = 5000
embedding_dim = 128
sequence_length = 500

vectorize_layer = TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [7]:
# Make a text-only dataset (no labels) and call adapt
text_ds = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

In [8]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [9]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [10]:
model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(4)])

In [11]:
model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True), 
    optimizer='adam', 
    metrics=['accuracy'])

In [13]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10)

Epoch 1/10
200/200 [==============================] - 7s 37ms/step - loss: 0.8932 - accuracy: 0.7330 - val_loss: 0.8661 - val_accuracy: 0.7525
Epoch 2/10
200/200 [==============================] - 7s 37ms/step - loss: 0.8110 - accuracy: 0.7588 - val_loss: 0.7982 - val_accuracy: 0.7638
Epoch 3/10
200/200 [==============================] - 7s 37ms/step - loss: 0.7442 - accuracy: 0.7758 - val_loss: 0.7440 - val_accuracy: 0.7731
Epoch 4/10
200/200 [==============================] - 7s 37ms/step - loss: 0.6883 - accuracy: 0.7894 - val_loss: 0.7002 - val_accuracy: 0.7781
Epoch 5/10
200/200 [==============================] - 8s 38ms/step - loss: 0.6418 - accuracy: 0.8067 - val_loss: 0.6639 - val_accuracy: 0.7912
Epoch 6/10
200/200 [==============================] - 8s 39ms/step - loss: 0.6016 - accuracy: 0.8183 - val_loss: 0.6343 - val_accuracy: 0.7969
Epoch 7/10
200/200 [==============================] - 7s 37ms/step - loss: 0.5640 - accuracy: 0.8347 - val_loss: 0.6087 - val_accuracy: 0.8000

In [14]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

250/250 [==============================] - 4s 17ms/step - loss: 0.6061 - accuracy: 0.7804
Loss:  0.6061021089553833
Accuracy:  0.7803750038146973
